In [326]:
import pandas as pd
from utils import *

In [327]:
# -- SALES_TARGET_DATA doeltabel --

# SALES_TARGET_DATA_ID
# SALES_TARGET_DATA_SALES-TARGET
# SALES_TARGET_DATA_PRODUCTIVITY_SCORE* (calculateSPS(): (UNIT_PRICE * QUANTITY) / SALES_TARGET)

# ORDER_DETAILS_UNIT_COST
# ORDER_DETAILS_UNIT_PRICE
# ORDER_DETAILS_UNIT_SALE_PRICE
# ORDER_DETAILS_QUANTITY

# SALES_STAFF_CODE
# SALES_BRANCH_CODE
# PRODUCT_NUMBER
# RETAILER_CODE

In [328]:
go_sales = load_db('source/go_sales.sqlite')
go_crm = load_db('source/go_crm.sqlite')

In [329]:
sales_target_data = load_table(go_sales, 'SALES_TARGETData').add_prefix('SALES_TARGET_DATA_')
order_header = load_table(go_sales, 'ORDER_HEADER').add_prefix('ORDER_HEADER_')
order_details = load_table(go_sales, 'ORDER_DETAILS').add_prefix('ORDER_DETAILS_')
sales_staff = load_table(go_sales, 'SALES_STAFF').add_prefix('SALES_STAFF_')
sales_branch = load_table(go_sales, 'SALES_BRANCH').add_prefix('SALES_BRANCH_')
product = load_table(go_sales, 'PRODUCT').add_prefix('PRODUCT_')

In [330]:
# Aggregate order
order = order_details.merge(order_header, left_on='ORDER_DETAILS_ORDER_NUMBER', right_on='ORDER_HEADER_ORDER_NUMBER')
order = order.merge(sales_staff, left_on='ORDER_HEADER_SALES_STAFF_CODE', right_on='SALES_STAFF_SALES_STAFF_CODE', how='left')
order


,ORDER_DETAILS_ORDER_DETAIL_CODE,ORDER_DETAILS_ORDER_NUMBER,ORDER_DETAILS_PRODUCT_NUMBER,ORDER_DETAILS_QUANTITY,ORDER_DETAILS_UNIT_COST,ORDER_DETAILS_UNIT_PRICE,ORDER_DETAILS_UNIT_SALE_PRICE,ORDER_HEADER_ORDER_NUMBER,ORDER_HEADER_RETAILER_NAME,ORDER_HEADER_RETAILER_SITE_CODE,...,SALES_STAFF_SALES_STAFF_CODE,SALES_STAFF_FIRST_NAME,SALES_STAFF_LAST_NAME,SALES_STAFF_POSITION_EN,SALES_STAFF_WORK_PHONE,SALES_STAFF_EXTENSION,SALES_STAFF_FAX,SALES_STAFF_EMAIL,SALES_STAFF_DATE_HIRED,SALES_STAFF_SALES_BRANCH_CODE
0,100000,8462,96,16,16.1,22.54,22.54,8462,Ocio y Aventura,334,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29
1,100001,9111,96,20,16.1,22.54,22.54,9111,Naranco de Bulnes,330,...,94,Yolanda,Torres,Level 3 Sales Representative,+35 94 322 3540,125,+35 91 548 1637,YTorres@grtd123.com,29-8-2011,29
2,100002,8451,96,24,16.1,22.54,22.54,8451,Ocio y Aventura,328,...,94,Yolanda,Torres,Level 3 Sales Representative,+35 94 322 3540,125,+35 91 548 1637,YTorres@grtd123.com,29-8-2011,29
3,100003,8453,96,18,16.1,22.54,22.54,8453,Ocio y Aventura,328,...,94,Yolanda,Torres,Level 3 Sales Representative,+35 94 322 3540,125,+35 91 548 1637,YTorres@grtd123.com,29-8-2011,29
4,100004,8439,96,20,16.1,22.54,22.54,8439,Naranco de Bulnes,326,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99995,8459,95,146,2.76,5.55,5.55,8459,Ocio y Aventura,333,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29
43059,99996,8457,95,172,2.76,5.55,5.55,8457,Ocio y Aventura,333,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29
43060,99997,9267,95,192,2.76,5.55,5.55,9267,Ocio y Aventura,334,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29
43061,99998,8441,95,192,2.76,5.55,5.55,8441,Naranco de Bulnes,326,...,95,Tomás,Iglesias,Level 2 Sales Representative,+35 94 322 3540,325,+35 91 548 1637,TIglesias@grtd123.com,19-2-2014,29


In [331]:
order['ORDER_HEADER_ORDER_DATE'] = pd.to_datetime(order['ORDER_HEADER_ORDER_DATE'])
order['YEAR_MONTH_YEAR'] = order['ORDER_HEADER_ORDER_DATE'].dt.year.astype(int)
order['YEAR_MONTH_MONTH'] = order['ORDER_HEADER_ORDER_DATE'].dt.month.astype(int)
order['ORDER_DETAILS_PRODUCT_NUMBER'] = order['ORDER_DETAILS_PRODUCT_NUMBER'].astype(int)

# Remove original date columns
order = order.drop(columns=['ORDER_HEADER_ORDER_DATE', 'ORDER_HEADER_ORDER_NUMBER', 'ORDER_DETAILS_ORDER_DETAIL_CODE'])
order = order[['ORDER_DETAILS_PRODUCT_NUMBER', 'ORDER_DETAILS_QUANTITY', 'ORDER_DETAILS_UNIT_COST', 'ORDER_DETAILS_UNIT_PRICE', 'ORDER_DETAILS_UNIT_SALE_PRICE', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH', 'SALES_STAFF_SALES_STAFF_CODE', 'SALES_STAFF_SALES_BRANCH_CODE']]

order['ORDER_DETAILS_QUANTITY'] = order['ORDER_DETAILS_QUANTITY'].astype(int)
order['ORDER_DETAILS_UNIT_COST'] = order['ORDER_DETAILS_UNIT_COST'].astype(float)
order['ORDER_DETAILS_UNIT_PRICE'] = order['ORDER_DETAILS_UNIT_PRICE'].astype(float)
order['ORDER_DETAILS_UNIT_SALE_PRICE'] = order['ORDER_DETAILS_UNIT_SALE_PRICE'].astype(float)

# Calculate total cost, price and sale price
order['ORDER_DETAILS_TOTAL_COST'] = order['ORDER_DETAILS_QUANTITY'] * order['ORDER_DETAILS_UNIT_COST']
order['ORDER_DETAILS_TOTAL_PRICE'] = order['ORDER_DETAILS_QUANTITY'] * order['ORDER_DETAILS_UNIT_PRICE']
order['ORDER_DETAILS_TOTAL_SALE_PRICE'] = order['ORDER_DETAILS_QUANTITY'] * order['ORDER_DETAILS_UNIT_SALE_PRICE']

# Remove original columns
order = order.drop(columns=['ORDER_DETAILS_UNIT_COST', 'ORDER_DETAILS_UNIT_PRICE', 'ORDER_DETAILS_UNIT_SALE_PRICE'])

# Aggregate order
order = order.groupby(['SALES_STAFF_SALES_STAFF_CODE', 'ORDER_DETAILS_PRODUCT_NUMBER', 'SALES_STAFF_SALES_BRANCH_CODE', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH']).agg(
    ORDER_DETAILS_TOTAL_COST=('ORDER_DETAILS_TOTAL_COST', 'sum'),
    ORDER_DETAILS_TOTAL_PRICE=('ORDER_DETAILS_TOTAL_PRICE', 'sum'),
    ORDER_DETAILS_TOTAL_SALE_PRICE=('ORDER_DETAILS_TOTAL_SALE_PRICE', 'sum'),
    ORDER_DETAILS_QUANTITY=('ORDER_DETAILS_QUANTITY', 'sum')
).reset_index()

order['ORDER_DETAILS_QUANTITY'] = order['ORDER_DETAILS_QUANTITY'].astype(int)
order['YEAR_MONTH_YEAR'] = order['YEAR_MONTH_YEAR'].astype(int)
order['YEAR_MONTH_MONTH'] = order['YEAR_MONTH_MONTH'].astype(int)
order


,SALES_STAFF_SALES_STAFF_CODE,ORDER_DETAILS_PRODUCT_NUMBER,SALES_STAFF_SALES_BRANCH_CODE,YEAR_MONTH_YEAR,YEAR_MONTH_MONTH,ORDER_DETAILS_TOTAL_COST,ORDER_DETAILS_TOTAL_PRICE,ORDER_DETAILS_TOTAL_SALE_PRICE,ORDER_DETAILS_QUANTITY
0,100,1,31,2021,3,256.64,385.92,385.92,64
1,100,1,31,2021,8,120.30,180.90,180.90,30
2,100,1,31,2022,6,384.00,576.00,546.24,96
3,100,1,31,2022,7,248.00,372.00,352.78,62
4,100,1,31,2022,9,472.00,708.00,671.42,118
...,...,...,...,...,...,...,...,...,...
37584,99,114,31,2022,9,160.00,400.00,364.00,2
37585,99,115,31,2020,7,74.76,149.94,149.94,14
37586,99,115,31,2021,6,110.22,220.66,203.94,22
37587,99,115,31,2021,7,120.24,240.72,222.48,24


In [332]:
# Merge with sales_target_data
sales_target_data['SALES_TARGET_DATA_SALES_YEAR'] = sales_target_data['SALES_TARGET_DATA_SALES_YEAR'].astype(int)
sales_target_data['SALES_TARGET_DATA_SALES_PERIOD'] = sales_target_data['SALES_TARGET_DATA_SALES_PERIOD'].astype(int)
sales_target_data['SALES_TARGET_DATA_PRODUCT_NUMBER'] = sales_target_data['SALES_TARGET_DATA_PRODUCT_NUMBER'].astype(int)
sales_target_data['SALES_TARGET_DATA_SALES_TARGET'] = sales_target_data['SALES_TARGET_DATA_SALES_TARGET'].astype(int)


result = sales_target_data.merge(order, left_on=['SALES_TARGET_DATA_SALES_STAFF_CODE', 'SALES_TARGET_DATA_PRODUCT_NUMBER', 'SALES_TARGET_DATA_SALES_YEAR', 'SALES_TARGET_DATA_SALES_PERIOD'], right_on=['SALES_STAFF_SALES_STAFF_CODE', 'ORDER_DETAILS_PRODUCT_NUMBER', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH'], how='left')
result

,SALES_TARGET_DATA_Id,SALES_TARGET_DATA_SALES_STAFF_CODE,SALES_TARGET_DATA_SALES_YEAR,SALES_TARGET_DATA_SALES_PERIOD,SALES_TARGET_DATA_RETAILER_NAME,SALES_TARGET_DATA_PRODUCT_NUMBER,SALES_TARGET_DATA_SALES_TARGET,SALES_TARGET_DATA_RETAILER_CODE,SALES_STAFF_SALES_STAFF_CODE,ORDER_DETAILS_PRODUCT_NUMBER,SALES_STAFF_SALES_BRANCH_CODE,YEAR_MONTH_YEAR,YEAR_MONTH_MONTH,ORDER_DETAILS_TOTAL_COST,ORDER_DETAILS_TOTAL_PRICE,ORDER_DETAILS_TOTAL_SALE_PRICE,ORDER_DETAILS_QUANTITY
0,1,6,2021,5,Altitudes extrêmes,53,489,33,6,53.0,6,2021.0,5.0,507.60,710.82,710.82,18.0
1,2,6,2021,5,Altitudes extrêmes,55,666,33,6,55.0,6,2021.0,5.0,483.60,967.20,967.20,124.0
2,3,6,2021,5,Altitudes extrêmes,56,682,33,6,56.0,6,2021.0,5.0,494.74,991.22,991.22,58.0
3,4,6,2021,5,Camping Sauvage,76,3193,35,6,76.0,6,2021.0,5.0,2730.64,4642.32,4642.32,116.0
4,5,6,2021,5,Camping Sauvage,85,7236,35,6,85.0,6,2021.0,5.0,7795.48,13252.80,9933.88,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39525,39526,83,2020,8,Golf Shop Jiro,107,4683,129,83,107.0,25,2020.0,8.0,1236.10,2719.44,2719.44,2.0
39526,39527,83,2021,3,Japan Sports Youhinten,33,497,130,83,33.0,25,2021.0,3.0,327.60,491.40,491.40,18.0
39527,39528,83,2021,3,Kitanaka Tozanyouhinten,43,1564,133,83,43.0,25,2021.0,3.0,1012.08,1518.16,1518.16,8.0
39528,39529,83,2021,3,Kitanaka Tozanyouhinten,60,600,133,83,60.0,25,2021.0,3.0,296.70,593.40,593.40,30.0


In [333]:
# Cleanup duplicate columns
result = result[[
    'SALES_TARGET_DATA_Id',
    'SALES_TARGET_DATA_SALES_TARGET',
    'ORDER_DETAILS_TOTAL_COST',
    'ORDER_DETAILS_TOTAL_PRICE',
    'ORDER_DETAILS_TOTAL_SALE_PRICE',
    'ORDER_DETAILS_QUANTITY',
    'SALES_TARGET_DATA_SALES_YEAR',
    'SALES_TARGET_DATA_SALES_PERIOD',
    'SALES_TARGET_DATA_SALES_STAFF_CODE',
    'SALES_STAFF_SALES_BRANCH_CODE',
    'SALES_TARGET_DATA_PRODUCT_NUMBER',
    'SALES_TARGET_DATA_RETAILER_CODE',
]]

# Rename columns
result = result.rename(columns={
    'SALES_TARGET_DATA_Id': 'SALES_TARGET_DATA_ID',
    'ORDER_DETAILS_TOTAL_COST': 'ORDER_DETAILS_TOTAL_COST',
    'ORDER_DETAILS_TOTAL_PRICE': 'ORDER_DETAILS_TOTAL_PRICE',
    'ORDER_DETAILS_TOTAL_SALE_PRICE': 'ORDER_DETAILS_TOTAL_SALE_PRICE',
    'ORDER_DETAILS_QUANTITY': 'ORDER_DETAILS_QUANTITY',
    'SALES_TARGET_DATA_SALES_YEAR': 'YEAR_MONTH_YEAR',
    'SALES_TARGET_DATA_SALES_PERIOD': 'YEAR_MONTH_MONTH',
})

# Rename foreign key columns
result = result.rename(columns={
    'SALES_TARGET_DATA_SALES_STAFF_CODE': 'SALES_STAFF_CODE',
    'SALES_STAFF_SALES_BRANCH_CODE': 'SALES_BRANCH_CODE',
    'SALES_TARGET_DATA_PRODUCT_NUMBER': 'PRODUCT_PRODUCT_NUMBER',
})

result['YEAR_MONTH_YEAR'] = result['YEAR_MONTH_YEAR'].astype(int)
result['YEAR_MONTH_MONTH'] = result['YEAR_MONTH_MONTH'].astype(int)

result

,SALES_TARGET_DATA_ID,SALES_TARGET_DATA_SALES_TARGET,ORDER_DETAILS_TOTAL_COST,ORDER_DETAILS_TOTAL_PRICE,ORDER_DETAILS_TOTAL_SALE_PRICE,ORDER_DETAILS_QUANTITY,YEAR_MONTH_YEAR,YEAR_MONTH_MONTH,SALES_STAFF_CODE,SALES_BRANCH_CODE,PRODUCT_PRODUCT_NUMBER,SALES_TARGET_DATA_RETAILER_CODE
0,1,489,507.60,710.82,710.82,18.0,2021,5,6,6,53,33
1,2,666,483.60,967.20,967.20,124.0,2021,5,6,6,55,33
2,3,682,494.74,991.22,991.22,58.0,2021,5,6,6,56,33
3,4,3193,2730.64,4642.32,4642.32,116.0,2021,5,6,6,76,35
4,5,7236,7795.48,13252.80,9933.88,44.0,2021,5,6,6,85,35
...,...,...,...,...,...,...,...,...,...,...,...,...
39525,39526,4683,1236.10,2719.44,2719.44,2.0,2020,8,83,25,107,129
39526,39527,497,327.60,491.40,491.40,18.0,2021,3,83,25,33,130
39527,39528,1564,1012.08,1518.16,1518.16,8.0,2021,3,83,25,43,133
39528,39529,600,296.70,593.40,593.40,30.0,2021,3,83,25,60,133
